# Detecting Jumps from Fitbit

In [7]:
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os
import sys
sys.path.append('../')

import pandas as pd
pd.set_option('display.max_columns', 200)

import src.generate_report as report_gen

## Accessing the Fitbit Data

In [9]:
import gather_keys_oauth2 as Oauth2
import fitbit
from datetime import datetime
from src.settings import FITBIT_CLIENT_ID, FITBIT_CLIENT_SECRET

### Setting Up the Server

In [10]:
server=Oauth2.OAuth2Server(FITBIT_CLIENT_ID, FITBIT_CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(FITBIT_CLIENT_ID,FITBIT_CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[06/Aug/2022:17:54:44] ENGINE Listening for SIGTERM.
[06/Aug/2022:17:54:44] ENGINE Listening for SIGHUP.
[06/Aug/2022:17:54:44] ENGINE Listening for SIGUSR1.
[06/Aug/2022:17:54:44] ENGINE Bus STARTING
[06/Aug/2022:17:54:44] ENGINE Started monitor thread 'Autoreloader'.
[06/Aug/2022:17:54:44] ENGINE Serving on http://127.0.0.1:8080
[06/Aug/2022:17:54:44] ENGINE Bus STARTED


127.0.0.1 - - [06/Aug/2022:17:54:46] "GET /?code=ff69134fec609e905c12a098f1b095fe6e9c731e&state=czZVJ8J7Z9TNio9hl8RJ0xEyf34usa HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15"


[06/Aug/2022:17:54:47] ENGINE Bus STOPPING
[06/Aug/2022:17:54:47] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[06/Aug/2022:17:54:47] ENGINE Stopped thread 'Autoreloader'.
[06/Aug/2022:17:54:47] ENGINE Bus STOPPED
[06/Aug/2022:17:54:47] ENGINE Bus EXITING
[06/Aug/2022:17:54:47] ENGINE Bus EXITED
[06/Aug/2022:17:54:47] ENGINE Waiting for child threads to terminate...


### Pull Data

In [38]:
def get_fb_data(y, m, d, resource):
    """
    Gets the fb data as a DataFrame from the given day
    """
    date = datetime(y, m, d)
    try:
        raw = auth2_client.intraday_time_series(f'activities/{resource}', date, detail_level='1sec')
    except Exception as e:
        print(e)
        raw = auth2_client.intraday_time_series(f'activities/{resource}', date, detail_level='1min')

    data =  pd.DataFrame(raw[f"activities-{resource}-intraday"]["dataset"])
    ts = []
    for t in data["time"]:
        t = pd.to_datetime(t)
        ts.append(datetime(y,m,d,t.hour,t.minute,t.second))
        
    data["time"] = ts
    return data.set_index("time")

### Comparing Logged Jumps to Fitbit Floor Data
Through a set of experiments, we can see if Fitbit data can be used to measure jumping. Can we find when people jump by looking at floors and elevation?

#### Floors vs Elevation
What is the relationship?


In [41]:
temp_floors = get_fb_data(2022,8,5,"floors")
temp_elevation = get_fb_data(2022,8,5,"elevation")
floor_vs_elevation = temp_elevation[datetime(2022,8,5,7,15):][:30].merge(temp_floors[datetime(2022,8,5,7,15):][:30],left_index=True,right_index=True)
floor_vs_elevation


One second interval is not supported for selected data type
One second interval is not supported for selected data type


,value_x,value_y
time,,
2022-08-05 07:15:00,0,0
2022-08-05 07:16:00,0,0
2022-08-05 07:17:00,0,0
2022-08-05 07:18:00,10,1
2022-08-05 07:19:00,10,1
2022-08-05 07:20:00,0,0
2022-08-05 07:21:00,0,0
2022-08-05 07:22:00,10,1
2022-08-05 07:23:00,10,1


Seems that floors are just elevation divided by 10

#### "Point" Jumps
For lack of a better word, "Point" jumps refer to a single instance of one or two jumps conducting at the turn of the minute. Here is the log:

| Time (Approximate) | Activity |
| --- | --- |
|17:32 | 1 Jump |
|17:33 | Nothing |
|17:34 | 2 Jump |
|17:35 | Nothing |
|17:36 | 2 Jump |
|17:37 | Nothing |
|17:38 | 1 Jump |


In [42]:
jump_data = get_fb_data(2022,8,6,"floors")
point_jump_data = jump_data[datetime(2022,8,6,17,30):datetime(2022,8,6,17,40)]
point_jump_data["jumps"] = [0,0,1,0,2,0,2,0,1,0,0]
point_jump_data

One second interval is not supported for selected data type


,value,jumps
time,,
2022-08-06 17:30:00,0,0
2022-08-06 17:31:00,1,0
2022-08-06 17:32:00,0,1
2022-08-06 17:33:00,0,0
2022-08-06 17:34:00,0,2
2022-08-06 17:35:00,0,0
2022-08-06 17:36:00,0,2
2022-08-06 17:37:00,0,0
2022-08-06 17:38:00,0,1


We don't seem to capture the jumps although perhaps some more experimens will lend a better view.

---